# CSCI 4253 / 5253 - Lab #3 - Patent Problem with SQL - SOLUTION
<div>
 <h2> CSCI 4253 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/" WIDTH=50 ALIGN="right"/> </h2>
</div>

In this assignment, we're going to solve a problem for which you'll also see the solution in Hadoop and then implement a PySpark solution. We have two databases
* One contains information about patents
* One contains information about patent citations (one patent citing the work of another)
The problem we're going to solve is augmenting the original patent data to include the number of *co-state citations*. In other words, if patent X was issued to someone in Colorado and patent Y was also issued to someone in Colorado and X cites Y, then this is a co-state citation.

The easiest way to solve this is to build an intermediary product from the patent citations table. That table contains `CITING` and `CITED` columns; you would augment it by adding `CITING_STATE` and `CITED_STATE`. Then, it becomes fairly simple to filter out all the cases where those states don't match. You can then use an sql `GROUP BY` to `COUNT(*)` all the co-state citations for a given patent and then join that with the original patents table resulting in an augmented table.

The final results for the first 13 words, sorted in descending order by co-state citations looks like the following
![this final output](final-output.png)

The challenge is going to be that I want you do to this *a single SQL query*. We're assuming you've learned some SQL in a previous life and if not, [now is a great time to learn](https://www.sqlitetutorial.net/). Even if you've done basic SQL, you'll probably need to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).


## Logistics

We're going to be using the SQLite3 system which runs entirely from a file (no server needed). The `Makefile` contains commands to download the raw data as ZIP files.

We can run shell commands in our notebook using [builtin "magic" commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). You need to run this at least once prior to starting the lab to make certain you have the files and have created the `patents.sq3` database file. It will take a few minutes to complete & you should see that the `patents.sq3` file is about 645MBytes in size.

In [1]:
%%bash
unzip -l acite75_99.zip | head -20
unzip -l apat63_99.zip | head -20

Archive:  acite75_99.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
264075431  10-09-2001 11:47   cite75_99.txt
---------                     -------
264075431                     1 file
Archive:  apat63_99.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
236903179  10-09-2001 12:00   apat63_99.txt
---------                     -------
236903179                     1 file


In [1]:
%%bash
make
rm patents.sq3
unzip -p acite75_99.zip > /tmp/citations.csv
unzip -p apat63_99.zip > /tmp/patents.csv
# Then import from the files
sqlite3 patents.sq3 <<EOF
.mode csv
.import /tmp/citations.csv citations
.import /tmp/patents.csv patents
EOF
ls -l patents.sq3

make: Nothing to be done for `all'.
-rw-r--r--@ 1 sebastianricke  staff  649641984 Jan 29 16:40 patents.sq3


We'll use another "magic" to run SQL queries in notebook cells. The following will load the SQL extension and connect to the `patents.sq3` file.

In [2]:
%load_ext sql
%sql sqlite:///patents.sq3
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

Following this, we can run individual SQL queries and see the result by putting `%%sql` at the front of a cell. If you don't have that, you'll be running Python code.

So, for example, we can examine our two raw database tables.

In [5]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///patents.sq3
Done.


name
citations
patents


In [3]:
%%sql
select * from patents limit 5;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD
3070801,1963,1096,,BE,,,1,,269,6,69,,1,,0,,,,,,,
3070802,1963,1096,,US,TX,,1,,2,6,63,,0,,,,,,,,,
3070803,1963,1096,,US,IL,,1,,2,6,63,,9,,0.3704,,,,,,,
3070804,1963,1096,,US,OH,,1,,2,6,63,,3,,0.6667,,,,,,,
3070805,1963,1096,,US,CA,,1,,2,6,63,,1,,0,,,,,,,


In [5]:
%%sql
select * from citations limit 5;

 * sqlite:///patents.sq3
Done.


CITING,CITED
3858241,956203
3858241,1324234
3858241,3398406
3858241,3557384
3858241,3634889


If you want to create indexes over various fields, go ahead. It shouldn't affect the correctness of your results but may affect the performance.

## Steps to the full solution

In order to determine when a *cited* patent and a *citing* patent are from the same state,
we're going to need to produce a series of tables that combine information from the citations and the patents tables.

We can use a simple left-outer join (or just **JOIN**) to get the information for one or the other column in the citations table. For example, we can determine the state for *cited* patents using this join:

In [ ]:
%%sql
SELECT CITED, patents.POSTATE as CITED_POSTATE, CITING
FROM citations JOIN patents ON CITED = PATENT
LIMIT 5;

Then, you'll need to do the same for the `CITING` column as well. As mentioned earlier, you may want to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).

## Your solution

Eneter your solution as a single SQL query below:

In [10]:
%%sql
SELECT 
    *,
    COALESCE(co_state.CO_STATE_CITATIONS, 0) as CO_STATE_CITATIONS
FROM patents p
LEFT JOIN (
    SELECT 
        citations.CITING as PATENT,
        COUNT(*) as CO_STATE_CITATIONS
    FROM citations
    JOIN patents as p1 ON citations.CITED = p1.PATENT
    JOIN patents as p2 ON citations.CITING = p2.PATENT
    WHERE p1.POSTATE = p2.POSTATE
        AND p1.POSTATE IS NOT NULL
        AND p2.POSTATE IS NOT NULL
    GROUP BY citations.CITING
) as co_state ON p.PATENT = co_state.PATENT
WHERE p.POSTATE IS NOT NULL AND p.POSTATE != ''
ORDER BY CO_STATE_CITATIONS DESC
LIMIT 13;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,PATENT_1,CO_STATE_CITATIONS,CO_STATE_CITATIONS_1
5959466,1999,14515,1997,US,CA,5310,2,,326,4,46,159,0,1,,0.6186,,4.8868,0.0455,0.044,,,5959466,125,125
5983822,1999,14564,1998,US,TX,569900,2,,114,5,55,200,0,0.995,,0.7201,,12.45,0,0,,,5983822,103,103
6008204,1999,14606,1998,US,CA,749584,2,,514,3,31,121,0,1,,0.7415,,5,0.0085,0.0083,,,6008204,100,100
5952345,1999,14501,1997,US,CA,749584,2,,514,3,31,118,0,1,,0.7442,,5.1102,0,0,,,5952345,98,98
5958954,1999,14515,1997,US,CA,749584,2,,514,3,31,116,0,1,,0.7397,,5.181,0,0,,,5958954,96,96
5998655,1999,14585,1998,US,CA,,1,,560,1,14,114,0,1,,0.7387,,5.1667,,,,,5998655,96,96
5936426,1999,14466,1997,US,CA,5310,2,,326,4,46,178,0,1,,0.58,,11.2303,0.0765,0.073,,,5936426,94,94
5739256,1998,13983,1995,US,CA,70060,2,15,528,1,15,453,0,1,,0.8232,,15.1104,0.1124,0.1082,,,5739256,90,90
5913855,1999,14417,1997,US,CA,733846,2,,606,3,32,242,0,1,,0.7403,,8.3595,0,0,,,5913855,90,90
5925042,1999,14445,1997,US,CA,733846,2,,606,3,32,242,0,1,,0.7382,,8.3471,0,0,,,5925042,90,90
